# Mitandao ya neva ya kurudia

Katika moduli iliyopita, tulijifunza uwakilishi wa kina wa semantiki ya maandishi. Usanifu ambao tumekuwa tukitumia unakamata maana ya jumla ya maneno katika sentensi, lakini hauzingatii **mpangilio** wa maneno, kwa sababu operesheni ya kujumlisha inayofuata embeddings huondoa taarifa hii kutoka kwa maandishi ya asili. Kwa kuwa mifano hii haiwezi kuwakilisha mpangilio wa maneno, haiwezi kutatua kazi ngumu zaidi au zenye utata kama vile uzalishaji wa maandishi au kujibu maswali.

Ili kukamata maana ya mlolongo wa maandishi, tutatumia usanifu wa mtandao wa neva unaoitwa **mtandao wa neva wa kurudia**, au RNN. Tunapotumia RNN, tunapitisha sentensi yetu kupitia mtandao tokeni moja kwa wakati, na mtandao huzalisha **hali fulani**, ambayo tunapitisha tena kwa mtandao pamoja na tokeni inayofuata.

![Picha inayoonyesha mfano wa uzalishaji wa mtandao wa neva wa kurudia.](../../../../../translated_images/sw/rnn.27f5c29c53d727b5.webp)

Kwa kuzingatia mlolongo wa tokeni za ingizo $X_0,\dots,X_n$, RNN huunda mlolongo wa vizuizi vya mtandao wa neva, na hufundisha mlolongo huu kutoka mwanzo hadi mwisho kwa kutumia backpropagation. Kila kizuizi cha mtandao huchukua jozi $(X_i,S_i)$ kama ingizo, na huzalisha $S_{i+1}$ kama matokeo. Hali ya mwisho $S_n$ au matokeo $Y_n$ huingia kwenye classifier ya mstari ili kutoa matokeo. Vizuizi vyote vya mtandao vinashiriki uzito sawa, na hufundishwa kutoka mwanzo hadi mwisho kwa kutumia mchakato mmoja wa backpropagation.

> Mchoro hapo juu unaonyesha mtandao wa neva wa kurudia katika umbo lililofunguliwa (kushoto), na katika uwakilishi wa kurudia ulio kompakt (kulia). Ni muhimu kutambua kwamba Seli zote za RNN zina **uzito unaoweza kushirikishwa**.

Kwa sababu vekta za hali $S_0,\dots,S_n$ zinapitishwa kupitia mtandao, RNN inaweza kujifunza utegemezi wa mlolongo kati ya maneno. Kwa mfano, wakati neno *not* linapojitokeza mahali fulani katika mlolongo, linaweza kujifunza kukanusha vipengele fulani ndani ya vekta ya hali.

Ndani, kila seli ya RNN ina matriksi mawili ya uzito: $W_H$ na $W_I$, na upendeleo $b$. Katika kila hatua ya RNN, kwa kuzingatia ingizo $X_i$ na hali ya ingizo $S_i$, hali ya matokeo huhesabiwa kama $S_{i+1} = f(W_H\times S_i + W_I\times X_i+b)$, ambapo $f$ ni kazi ya uanzishaji (mara nyingi $\tanh$).

> Kwa matatizo kama uzalishaji wa maandishi (ambayo tutashughulikia katika kitengo kinachofuata) au tafsiri ya mashine, tunataka pia kupata thamani ya matokeo katika kila hatua ya RNN. Katika kesi hii, kuna pia matriksi mengine $W_O$, na matokeo huhesabiwa kama $Y_i=f(W_O\times S_i+b_O)$.

Hebu tuone jinsi mitandao ya neva ya kurudia inaweza kutusaidia kuainisha seti yetu ya data ya habari.

> Kwa mazingira ya sandbox, tunahitaji kuendesha seli ifuatayo ili kuhakikisha maktaba inayohitajika imewekwa, na data imepakiwa mapema. Ikiwa unafanya kazi kwa ndani, unaweza kuruka seli ifuatayo.


In [1]:
import sys
!{sys.executable} -m pip install --quiet tensorflow_datasets==4.4.0
!cd ~ && wget -q -O - https://mslearntensorflowlp.blob.core.windows.net/data/tfds-ag-news.tgz | tar xz

In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

# We are going to be training pretty large models. In order not to face errors, we need
# to set tensorflow option to grow GPU memory allocation when required
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

ds_train, ds_test = tfds.load('ag_news_subset').values()

Wakati wa kufundisha mifano mikubwa, ugawaji wa kumbukumbu ya GPU unaweza kuwa tatizo. Pia tunaweza kuhitaji kujaribu ukubwa tofauti wa minibatch, ili data iweze kutoshea kwenye kumbukumbu ya GPU yetu, lakini mafunzo yawe ya haraka vya kutosha. Ikiwa unaendesha msimbo huu kwenye mashine yako ya GPU, unaweza kujaribu kurekebisha ukubwa wa minibatch ili kuharakisha mafunzo.

> **Note**: Toleo fulani za madereva ya NVidia zinajulikana kwa kutotoa kumbukumbu baada ya kufundisha mfano. Tunaendesha mifano kadhaa katika daftari hili, na inaweza kusababisha kumbukumbu kuisha katika mipangilio fulani, hasa ikiwa unafanya majaribio yako mwenyewe kama sehemu ya daftari hilo hilo. Ikiwa utakutana na makosa ya ajabu unapoanza kufundisha mfano, unaweza kutaka kuwasha upya kernel ya daftari.


In [3]:
batch_size = 16
embed_size = 64

## Kainishi Rahisi ya RNN

Katika hali ya RNN rahisi, kila kitengo cha kurudia ni mtandao rahisi wa mstari, ambao unachukua vekta ya pembejeo na vekta ya hali, na kutoa vekta mpya ya hali. Katika Keras, hii inaweza kuwakilishwa na safu ya `SimpleRNN`.

Ingawa tunaweza kupitisha tokeni zilizokodishwa moja kwa moja kwa safu ya RNN, hii si wazo zuri kwa sababu ya ukubwa wao wa juu. Kwa hivyo, tutatumia safu ya embedding kupunguza ukubwa wa vekta za maneno, ikifuatiwa na safu ya RNN, na hatimaye classifier ya `Dense`.

> **Note**: Katika hali ambapo ukubwa si mkubwa sana, kwa mfano wakati wa kutumia tokeni za kiwango cha herufi, inaweza kuwa na maana kupitisha tokeni zilizokodishwa moja kwa moja kwenye seli ya RNN.


In [4]:
vocab_size = 20000

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=vocab_size,
    input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, embed_size),
    keras.layers.SimpleRNN(16),
    keras.layers.Dense(4,activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, None)              0         
_________________________________________________________________
embedding (Embedding)        (None, None, 64)          1280000   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 16)                1296      
_________________________________________________________________
dense (Dense)                (None, 4)                 68        
Total params: 1,281,364
Trainable params: 1,281,364
Non-trainable params: 0
_________________________________________________________________


> **Note:** Tunatumia safu ya embedding isiyofunzwa hapa kwa urahisi, lakini kwa matokeo bora tunaweza kutumia safu ya embedding iliyofunzwa awali kwa kutumia Word2Vec, kama ilivyoelezwa katika sehemu iliyopita. Itakuwa zoezi zuri kwako kubadilisha msimbo huu ili ufanye kazi na embeddings zilizofunzwa awali.

Sasa hebu tufunze RNN yetu. Kwa ujumla, RNN ni ngumu sana kufunza, kwa sababu mara seli za RNN zinapopanuliwa kulingana na urefu wa mlolongo, idadi ya tabaka zinazohusika katika backpropagation huwa kubwa sana. Kwa hivyo tunahitaji kuchagua kiwango kidogo cha kujifunza, na kufunza mtandao kwenye seti kubwa ya data ili kupata matokeo mazuri. Hii inaweza kuchukua muda mrefu sana, kwa hivyo kutumia GPU inapendekezwa.

Ili kuharakisha mambo, tutafunza tu mfano wa RNN kwenye vichwa vya habari vya habari, tukiacha maelezo. Unaweza kujaribu kufunza na maelezo na kuona kama unaweza kufanya mfano ufanye kazi.


In [5]:
def extract_title(x):
    return x['title']

def tupelize_title(x):
    return (extract_title(x),x['label'])

print('Training vectorizer')
vectorizer.adapt(ds_train.take(2000).map(extract_title))

Training vectorizer


In [6]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize_title).batch(batch_size),validation_data=ds_test.map(tupelize_title).batch(batch_size))

7500/7500 [==============================] - 82s 11ms/step - loss: 0.6629 - acc: 0.7623 - val_loss: 0.5559 - val_acc: 0.7995


> **Kumbuka** kwamba usahihi unaweza kuwa wa chini hapa, kwa sababu tunajifunza tu kwa vichwa vya habari vya habari.


## Kurudia mfuatano wa vigezo

Kumbuka kwamba safu ya `TextVectorization` itaongeza kiotomatiki alama za kujaza (pad tokens) kwenye mfuatano wa urefu tofauti katika kundi dogo. Inageuka kuwa alama hizo pia zinashiriki katika mafunzo, na zinaweza kufanya mchakato wa muundo kufikia matokeo kuwa mgumu zaidi.

Kuna mbinu kadhaa tunazoweza kutumia kupunguza kiasi cha kujaza. Mojawapo ni kupanga upya seti ya data kulingana na urefu wa mfuatano na kuunganisha mfuatano wote kulingana na ukubwa. Hii inaweza kufanyika kwa kutumia kazi ya `tf.data.experimental.bucket_by_sequence_length` (tazama [hati](https://www.tensorflow.org/api_docs/python/tf/data/experimental/bucket_by_sequence_length)).

Mbinu nyingine ni kutumia **masking**. Katika Keras, baadhi ya safu zinasaidia pembejeo ya ziada inayoonyesha ni alama zipi zinapaswa kuzingatiwa wakati wa mafunzo. Ili kuingiza masking kwenye muundo wetu, tunaweza ama kujumuisha safu tofauti ya `Masking` ([hati](https://keras.io/api/layers/core_layers/masking/)), au tunaweza kubainisha kipengele cha `mask_zero=True` kwenye safu yetu ya `Embedding`.

> **Note**: Mafunzo haya yatachukua takriban dakika 5 kukamilisha kipindi kimoja kwenye seti nzima ya data. Usisite kusitisha mafunzo wakati wowote ikiwa utapoteza uvumilivu. Kile unachoweza pia kufanya ni kupunguza kiasi cha data kinachotumika kwa mafunzo, kwa kuongeza kipengele `.take(...)` baada ya seti za data `ds_train` na `ds_test`.


In [7]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size,embed_size,mask_zero=True),
    keras.layers.SimpleRNN(16),
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

7500/7500 [==============================] - 371s 49ms/step - loss: 0.5401 - acc: 0.8079 - val_loss: 0.3780 - val_acc: 0.8822


Sasa kwa kuwa tunatumia masking, tunaweza kufundisha modeli kwa seti nzima ya data ya vichwa vya habari na maelezo.

> **Note**: Je, umeona kwamba tumekuwa tukitumia vectorizer iliyofundishwa kwa vichwa vya habari vya habari, na si mwili mzima wa makala? Inawezekana, hili linaweza kusababisha baadhi ya token kupuuzwa, kwa hivyo ni bora kufundisha upya vectorizer. Hata hivyo, huenda hili likawa na athari ndogo sana, kwa hivyo tutatumia vectorizer iliyofundishwa awali kwa urahisi.


## LSTM: Kumbukumbu ya muda mrefu na mfupi

Moja ya matatizo makuu ya RNNs ni **kupotea kwa gradients**. RNNs zinaweza kuwa ndefu sana, na zinaweza kuwa na ugumu wa kusambaza gradients hadi kwenye safu ya kwanza ya mtandao wakati wa backpropagation. Hali hii ikitokea, mtandao hauwezi kujifunza uhusiano kati ya tokeni za mbali. Njia moja ya kuepuka tatizo hili ni kuanzisha **usimamizi wa hali wazi** kwa kutumia **milango**. Miundo miwili maarufu inayotumia milango ni **kumbukumbu ya muda mrefu na mfupi** (LSTM) na **kitengo cha relay chenye milango** (GRU). Tutazungumzia LSTMs hapa.

![Picha inayoonyesha mfano wa seli ya kumbukumbu ya muda mrefu na mfupi](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

Mtandao wa LSTM umeandaliwa kwa namna inayofanana na RNN, lakini kuna hali mbili zinazopitishwa kutoka safu moja hadi nyingine: hali halisi $c$, na vector iliyofichwa $h$. Kwenye kila kitengo, vector iliyofichwa $h_{t-1}$ inaunganishwa na ingizo $x_t$, na pamoja zinadhibiti kinachotokea kwa hali $c_t$ na matokeo $h_{t}$ kupitia **milango**. Kila mlango una uanzishaji wa sigmoid (matokeo katika safu $[0,1]$), ambayo inaweza kufikiriwa kama mask ya biti wakati inazidishwa na vector ya hali. LSTMs zina milango ifuatayo (kutoka kushoto kwenda kulia kwenye picha hapo juu):
* **mlango wa kusahau** ambao huamua ni vipengele gani vya vector $c_{t-1}$ tunahitaji kusahau, na vipi kupitisha.
* **mlango wa ingizo** ambao huamua ni kiasi gani cha taarifa kutoka kwa vector ya ingizo na vector iliyofichwa ya awali kinapaswa kuingizwa kwenye vector ya hali.
* **mlango wa matokeo** ambao huchukua vector mpya ya hali na kuamua ni vipengele gani vya vector hiyo vitatumika kuzalisha vector mpya iliyofichwa $h_t$.

Vipengele vya hali $c$ vinaweza kufikiriwa kama bendera zinazoweza kuwashwa na kuzimwa. Kwa mfano, tunapokutana na jina *Alice* katika mlolongo, tunakisia kuwa linahusu mwanamke, na tunainua bendera katika hali inayoonyesha kuwa tuna nomino ya kike katika sentensi. Tunapokutana na maneno *na Tom*, tunainua bendera inayoonyesha kuwa tuna nomino ya wingi. Hivyo, kwa kudhibiti hali tunaweza kufuatilia mali za kisarufi za sentensi.

> **Note**: Hapa kuna rasilimali nzuri ya kuelewa undani wa LSTMs: [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) na Christopher Olah.

Ingawa muundo wa ndani wa seli ya LSTM unaweza kuonekana kuwa mgumu, Keras huficha utekelezaji huu ndani ya safu ya `LSTM`, kwa hivyo kitu pekee tunachohitaji kufanya katika mfano hapo juu ni kubadilisha safu ya kurudia:


In [8]:
model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, embed_size),
    keras.layers.LSTM(8),
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(8),validation_data=ds_test.map(tupelize).batch(8))

15000/15000 [==============================] - 188s 13ms/step - loss: 0.5692 - acc: 0.7916 - val_loss: 0.3441 - val_acc: 0.8870


## RNN za Mwelekeo Mbili na Tabaka Nyingi

Katika mifano yetu hadi sasa, mitandao ya kurudia hufanya kazi kutoka mwanzo wa mlolongo hadi mwisho. Hii inahisi kuwa ya kawaida kwetu kwa sababu inafuata mwelekeo sawa ambao tunasoma au kusikiliza hotuba. Hata hivyo, kwa hali zinazohitaji ufikiaji wa nasibu wa mlolongo wa pembejeo, ina mantiki zaidi kuendesha hesabu ya kurudia katika pande zote mbili. RNNs zinazowezesha hesabu katika pande zote mbili zinaitwa **RNN za mwelekeo mbili**, na zinaweza kuundwa kwa kuzungushia tabaka ya kurudia na tabaka maalum ya `Bidirectional`.

> **Note**: Tabaka ya `Bidirectional` hufanya nakala mbili za tabaka ndani yake, na kuweka mali ya `go_backwards` ya moja ya nakala hizo kuwa `True`, ikifanya iende katika mwelekeo kinyume na mlolongo.

Mitandao ya kurudia, iwe ya mwelekeo mmoja au mwelekeo mbili, huchukua mifumo ndani ya mlolongo, na kuihifadhi katika vekta za hali au kuzirudisha kama matokeo. Kama ilivyo kwa mitandao ya convolutional, tunaweza kujenga tabaka nyingine ya kurudia kufuatia ya kwanza ili kuchukua mifumo ya kiwango cha juu, iliyojengwa kutoka mifumo ya kiwango cha chini iliyotolewa na tabaka ya kwanza. Hii inatupeleka kwenye dhana ya **RNN ya tabaka nyingi**, ambayo inajumuisha mitandao miwili au zaidi ya kurudia, ambapo matokeo ya tabaka ya awali hupitishwa kwa tabaka inayofuata kama pembejeo.

![Picha inayoonyesha RNN ya Tabaka Nyingi ya kumbukumbu ya muda mrefu na mfupi](../../../../../translated_images/sw/multi-layer-lstm.dd975e29bb2a59fe.webp)

*Picha kutoka [chapisho hili zuri](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) na Fernando López.*

Keras inafanya ujenzi wa mitandao hii kuwa kazi rahisi, kwa sababu unahitaji tu kuongeza tabaka zaidi za kurudia kwenye modeli. Kwa tabaka zote isipokuwa ya mwisho, tunahitaji kubainisha parameter ya `return_sequences=True`, kwa sababu tunahitaji tabaka irudishe hali zote za kati, na siyo hali ya mwisho tu ya hesabu ya kurudia.

Hebu tujenge LSTM ya tabaka mbili ya mwelekeo mbili kwa tatizo letu la uainishaji.

> **Note**: Msimbo huu tena unachukua muda mrefu kukamilika, lakini unatupa usahihi wa juu zaidi ambao tumeona hadi sasa. Kwa hivyo labda inafaa kusubiri na kuona matokeo.


In [9]:
model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, 128, mask_zero=True),
    keras.layers.Bidirectional(keras.layers.LSTM(64,return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),    
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

5045/7500 [===================>..........] - ETA: 2:33 - loss: 0.3709 - acc: 0.8706

## RNNs kwa kazi nyingine

Hadi sasa, tumekuwa tukizingatia kutumia RNNs kuainisha mfuatano wa maandishi. Lakini zinaweza kushughulikia kazi nyingi zaidi, kama vile uzalishaji wa maandishi na tafsiri ya lugha — tutazingatia kazi hizo katika kitengo kijacho.



---

**Kanusho**:  
Hati hii imetafsiriwa kwa kutumia huduma ya tafsiri ya AI [Co-op Translator](https://github.com/Azure/co-op-translator). Ingawa tunajitahidi kuhakikisha usahihi, tafadhali fahamu kuwa tafsiri za kiotomatiki zinaweza kuwa na makosa au kutokuwa sahihi. Hati ya asili katika lugha yake ya awali inapaswa kuzingatiwa kama chanzo cha mamlaka. Kwa taarifa muhimu, tafsiri ya kitaalamu ya binadamu inapendekezwa. Hatutawajibika kwa kutoelewana au tafsiri zisizo sahihi zinazotokana na matumizi ya tafsiri hii.
